In [7]:
%load_ext autoreload
%autoreload 2
import os
from os.path import join, expanduser
from tqdm import tqdm
import pandas as pd
import joblib
import sys
import numpy as np
from copy import deepcopy
from neuro.features.feat_select import get_alphas
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
sparse_feats_dir = expanduser('~/mntv1/deep-fMRI/qa/sparse_feats_shared/')
os.listdir(sparse_feats_dir)
qa_sparse_feats_dir = join(
    sparse_feats_dir,
    # 'qa_embedder___qa_questions_version=v3_boostexamples___ensemble1')
    'qa_embedder___qa_questions_version=v3_boostexamples_merged___ensemble2')
# 'eng1000___qa_questions_version=v1___mistralai-Mistral-7B-Instruct-v0.2')
# alphas = np.logspace(0, -3, 20)
alphas = get_alphas('qa_embedder')
# alphas = get_alphas('eng1000')

In [26]:
seeds = range(5)
nonzeros = pd.DataFrame(index=seeds, columns=alphas).astype(float)
enets = pd.DataFrame(index=seeds, columns=alphas)
fnames = os.listdir(qa_sparse_feats_dir)
for seed in tqdm(seeds):
    for alpha in tqdm(alphas):
        template = f'seed={seed}___feature_selection_frac=0.50___feature_selection_alpha={alpha:.2e}.joblib'
        if template in fnames:
            coef_enet = joblib.load(join(qa_sparse_feats_dir, template))
            coef_enet_selected = deepcopy(
                np.any(np.abs(coef_enet) > 0, axis=0).squeeze())
            print(f'{coef_enet.shape=}')
            enets.loc[seed, alpha] = coef_enet_selected
            nonzeros.loc[seed, alpha] = coef_enet_selected.sum()
# template = f'seed={seed}___feature_selection_frac=0.50___feature_selection_alpha={feature_selection_alpha:.2e}.joblib'
# os.listdir(qa_sparse_feats_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/home/chansingh/mntv1/deep-fMRI/qa/sparse_feats_shared/qa_embedder___qa_questions_version=v3_boostexamples_merged___ensemble2'

In [14]:
# v3_boost_examples has 674, eng1000 has 985
nonzeros.columns.name = 'alpha'
nonzeros.index.name = 'seed'
# nonzeros.columns = np.arange(len(nonzeros.columns))
# enets.columns = nonzeros.columns
# nonzeros.columns = nonzeros.columns.round(4)

coefs_stable_dict = {}
coefs_all_dict = {}
# add row for fracs
for col in nonzeros.columns:
    coefs_list = enets[col]
    coefs_list = coefs_list[coefs_list.notna()]

    if len(coefs_list) > 0:
        # get fraction of times each element is True
        coefs_arr = np.vstack(coefs_list.values)
        coefs_all = coefs_arr.max(axis=0)
        coefs_arr = coefs_arr.min(axis=0)
        nonzeros_stable = np.sum(coefs_arr)
    else:
        nonzeros_stable = np.nan
    nonzeros.loc['stable', col] = nonzeros_stable
    coefs_all_dict[col] = deepcopy(coefs_all)
    coefs_stable_dict[col] = deepcopy(coefs_arr)

display(
    nonzeros
    .style
    .background_gradient(cmap='viridis', axis=None)
    .format('{:.0f}')
)

len(coefs_list)=5
len(coefs_list)=5
len(coefs_list)=5
len(coefs_list)=5
len(coefs_list)=5
len(coefs_list)=5
len(coefs_list)=5
len(coefs_list)=5
len(coefs_list)=5


alpha,0.483293,0.400000,0.335982,0.280000,0.233572,0.162378,0.112884,0.078476,0.054556
seed,,,,,,,,,
0,9,17,25,49,73,164,326,480,603
1,9,19,26,46,80,174,322,483,nan
2,10,18,27,45,85,171,332,476,602
3,8,18,27,44,80,167,344,472,599
4,9,17,27,47,77,164,327,485,nan
stable,8,14,20,31,51,120,249,406,579


### Top questions

In [22]:
questions = np.array(get_merged_questions_v3_boostexamples())

In [24]:
questions = np.array(get_questions('v3_boostexamples', full=True))
# questions = np.array(get_merged_questions_v3_boostexamples())
# print(len(questions))
qs_prev = []
for i in range(5):
    print(i, 'num questions:', len(
        questions[list(coefs_stable_dict.values())[i]]))

    print('----STABLE---')
    qs = sorted(questions[list(coefs_stable_dict.values())[i]].tolist())
    qs_unstable = sorted(questions[list(coefs_all_dict.values())[i]].tolist())
    display([q for q in qs if q not in qs_prev])
    for q in qs_prev:
        if not q in qs:
            print('[DROPPED]', q)

    print('----UNSTABLE---')
    display([q for q in qs_unstable if not q in qs])

    qs_prev = qs

0 num questions: 8
----STABLE---


['Does the input include a philosophical or reflective thought?',
 'Does the sentence contain a proper noun?',
 'Does the sentence describe a personal or social interaction that leads to a change or revelation?',
 'Does the sentence describe a physical action?',
 'Does the sentence describe a relationship between people?',
 'Does the sentence involve a description of physical environment or setting?',
 'Does the sentence involve the mention of a specific object or item?',
 'Does the sentence reference a specific location or place?']

----UNSTABLE---


['Does the sentence describe a personal reflection or thought?',
 'Does the sentence describe a visual experience or scene?',
 'Does the sentence involve a social or interpersonal interaction?']

1 num questions: 14
----STABLE---


...

['Does the input describe a problem or challenge?',
 'Does the input describe a sensory experience?',
 'Does the input discuss a societal issue or social justice topic?',
 'Does the input include a comparison or metaphor?',
 'Does the sentence convey a sense of urgency or haste?',
 'Does the sentence describe a moment of relief or resolution of tension?',
 'Does the sentence describe a specific sensation or feeling?',
 'Does the sentence describe an activity related to daily life or routine?',
 'Does the sentence express a sense of belonging or connection to a place or community?',
 'Does the sentence include a direct speech quotation?',
 'Does the sentence include a recounting of an impactful or emotional dialogue?',
 'Does the sentence include a specific sound or auditory description?',
 'Does the sentence include dialogue or thoughts directed towards another character?',
 'Does the sentence involve a decision-making process?',
 'Does the sentence involve a description of an interper

4 num questions: 51
----STABLE---


['Does the input discuss a societal issue or social justice topic?',
 'Does the sentence convey a decision or choice made by the narrator?',
 'Does the sentence convey a sense of urgency or haste?',
 "Does the sentence convey the narrator's sense of humor or wit?",
 'Does the sentence describe a moment of relief or resolution of tension?',
 'Does the sentence describe a specific sensation or feeling?',
 'Does the sentence express a sense of belonging or connection to a place or community?',
 'Does the sentence include a direct speech quotation?',
 'Does the sentence include a specific sound or auditory description?',
 'Does the sentence involve a discussion about personal or social values?',
 'Does the sentence involve a safety or security concern?',
 'Does the sentence involve an unexpected incident or accident?',
 'Does the sentence reference a specific time or date?',
 'Does the sentence use a unique or unusual word?',
 'Does the text describe a mode of communication?',
 "Is the sen

----UNSTABLE---


['Does the input contain a measurement?',
 'Does the input describe a moment of realization or epiphany?',
 'Does the input describe a sensory experience?',
 'Does the input discuss a health-related issue or wellness?',
 'Does the input include a comparison or metaphor?',
 'Does the input include a description of clothing?',
 'Does the sentence describe a character in a story?',
 'Does the sentence describe a health-related topic or concern?',
 'Does the sentence describe a physical sensation (e.g., touch, taste)?',
 'Does the sentence describe a routine activity?',
 'Does the sentence describe an activity related to daily life or routine?',
 'Does the sentence describe an experience of learning or gaining new knowledge?',
 'Does the sentence describe an indoor setting?',
 'Does the sentence express anger or frustration directed at a specific situation or person?',
 'Does the sentence include a conditional clause?',
 'Does the sentence include a recounting of an impactful or emotional 

# Manually clean up questions

In [ ]:
questions = np.array(get_questions('v3_boostexamples', full=True))

In [ ]:
df = pd.DataFrame(questions)
df.index.name = 'question_num'
df.rename(columns={0: 'question'}, inplace=True)
df['stable_idx'] = 10
coefs_stable_vals = list(coefs_stable_dict.values())
for i in range(len(coefs_stable_vals) - 1, -1, -1):
    df.loc[coefs_stable_vals[i], 'stable_idx'] = i
df = df.sort_values(by='stable_idx')

# display full questions no truncation
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(df)

In [ ]:
df = df.sort_index()

In [ ]:
df

In [ ]:
s = ''
for i in range(len(df)):
    s += f'{i}. {questions[i]}\n'

In [ ]:
s

In [ ]:
'''Below is a numbered list of questions. Group and list any questions (along with their numbers) that are very similar to each other'''

# Select questions to run with gpt4

In [ ]:
d = df[df.stable_idx <= 4]

In [ ]:
d

In [ ]:
# drop anything that was merged
from neuro.features.questions.merge_v3_boostexamples import DICT_MERGE_V3_BOOSTEXAMPLES
questions_to_drop = [k for k in sum(DICT_MERGE_V3_BOOSTEXAMPLES.values(), [
]) if not k in DICT_MERGE_V3_BOOSTEXAMPLES]
d = d[~d.question.isin(questions_to_drop)]
d.shape

In [ ]:
d.question.values